### Imports

In [12]:
import sys
sys.path.insert(1, "../gapfilling/scripts/")

In [13]:
# loading gap filling module
from Gapfilling import *
from modelseed_gapfilling import *
import random

In [14]:
from cobra.flux_analysis import flux_variability_analysis

### New functions

In [15]:
def make_model_copies(model, n = 10):
    d = {}
    N = n + 1
    for i in range(1, N):
        d[str(model) + "_" + str(i)] = fake_copy(model)
    return d

In [119]:
def remove_random_reaction(reference, model_dict, reactions_list = [] , n = 1):
    """
    model_dict: dictionary of samples
    n: number of reactions to be removed from each model
    """
    n_samples = len(model_dict)
    N = n * n_samples
    
    if reactions_list == []:
        reactions = list(reference.reactions)
        Random = random.sample(reactions, N)
        random_reactions = [i.id for i in Random]
        
    else:
        random_reactions = random.sample(reactions_list, N)
        
    print(random_reactions)
    
    if N == n_samples:
        for i in range(len(model_dict)):
            list(model_dict.values())[i].remove_reactions([random_reactions[i]])
    else:
        c = 0
        for i in range(len(model_dict)):
            for j in range(c, c + n):
                list(model_dict.values())[i].remove_reactions([random_reactions[j]])
                c += 1
    return model_dict

### Testing
#### Loading models

In [65]:
e_coli = read_sbml_model("../experiment/models/references/iML1515.xml")
l_lactis = read_sbml_model("../experiment/models/references/iNF517.xml")
p_putida = read_sbml_model("../experiment/models/references/iJN1463.xml")

In [69]:
# these are the functions which bounds must be augmented in order to give the model degrees of freedom
l_lactis.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
4abz_e,EX_4abz_e,4.263E-07,7,0.00%
ade_e,EX_ade_e,0.00475,5,0.30%
ala__L_e,EX_ala__L_e,0.04,3,1.53%
arg__L_e,EX_arg__L_e,0.02,6,1.53%
asp__L_e,EX_asp__L_e,0.01,4,0.51%
cys__L_e,EX_cys__L_e,0.008785,3,0.34%
glc__D_e,EX_glc__D_e,1.095,6,83.71%
glu__L_e,EX_glu__L_e,0.05,5,3.19%
gly_e,EX_gly_e,0.002388,2,0.06%
gua_e,EX_gua_e,0.005057,5,0.32%


In [70]:
l_lactis.reactions.EX_glc__D_e

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x07f30e8292b80
Stoichiometry,glc__D_e <-- D-Glucose <--
GPR,
Lower bound,-2.12
Upper bound,-0.92


#### Checking objective functions

In [72]:
print("OBJECTIVE FUNCTIONS:\n")
print("E. coli:\n\n" + str(e_coli.objective) + "\n")
print("L. lactis:\n\n" + str(l_lactis.objective) + "\n")
print("P. putida:\n\n" + str(p_putida.objective) + "\n")

OBJECTIVE FUNCTIONS:

E. coli:

Maximize
1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685

L. lactis:

Maximize
1.0*BIOMASS_LLA - 1.0*BIOMASS_LLA_reverse_0796e

P. putida:

Maximize
1.0*BIOMASS_KT2440_WT3 - 1.0*BIOMASS_KT2440_WT3_reverse_d86d5



In [73]:
# Checking the fluxes of each model
print("FLUX VALUES:\n")
print("E. coli:\n\n" + str(e_coli.optimize().objective_value) + "\n")
print("L. lactis:\n\n" + str(l_lactis.optimize().objective_value) + "\n")
print("P. putida:\n\n" + str(p_putida.optimize().objective_value) + "\n")

FLUX VALUES:

E. coli:

0.8769972144269736

L. lactis:

0.04263460544337326

P. putida:

0.5861175448479816



The L.lactis model is pretty constrained. This is known to drastically affects gap filling, so we will remove these constrictions.

In [74]:
l_lactis.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
4abz_e,EX_4abz_e,4.263E-07,7,0.00%
ade_e,EX_ade_e,0.00475,5,0.30%
ala__L_e,EX_ala__L_e,0.04,3,1.53%
arg__L_e,EX_arg__L_e,0.02,6,1.53%
asp__L_e,EX_asp__L_e,0.01,4,0.51%
cys__L_e,EX_cys__L_e,0.008785,3,0.34%
glc__D_e,EX_glc__D_e,1.095,6,83.71%
glu__L_e,EX_glu__L_e,0.05,5,3.19%
gly_e,EX_gly_e,0.002388,2,0.06%
gua_e,EX_gua_e,0.005057,5,0.32%


In [75]:
# uptake (except Glc)
uptake = ['EX_4abz_e', 'EX_ade_e', 'EX_ala__L_e', 'EX_arg__L_e', 'EX_asp__L_e', 'EX_cys__L_e', 
          'EX_glu__L_e', 'EX_gly_e', 'EX_gua_e', 'EX_h2o_e', 'EX_his__L_e', 'EX_ile__L_e',
          'EX_leu__L_e', 'EX_lys__L_e', 'EX_met__L_e', 'EX_nac_e', 'EX_phe__L_e', 'EX_pi_e', 
          'EX_pnto__R_e', 'EX_ser__L_e', 'EX_thm_e', 'EX_thr__L_e', 'EX_thymd_e', 'EX_ura_e', 'EX_val__L_e']

# secretion
secretion = ['EX_ac_e', 'EX_acald_e', 'EX_co2_e', 'EX_etoh_e', 'EX_for_e', 'EX_gcald_e', 'EX_h2s_e', 
             'EX_h_e', 'EX_lac__L_e', 'EX_nh4_e', 'EX_pro__L_e']

In [76]:
for i in uptake:
    l_lactis.reactions.get_by_id(i).lower_bound = -30.0

In [77]:
l_lactis.reactions.EX_ile__L_e

Reaction identifier,EX_ile__L_e
Name,L-Isoleucine exchange
Memory address,0x07f30e82acdc0
Stoichiometry,ile__L_e <-- L-Isoleucine <--
GPR,
Lower bound,-30.0
Upper bound,-0.01


In [80]:
for i in secretion:
    l_lactis.reactions.get_by_id(i).upper_bound = 1000.0

In [81]:
l_lactis.reactions.EX_lac__L_e

Reaction identifier,EX_lac__L_e
Name,L-Lactate exchange
Memory address,0x07f30e8233fa0
Stoichiometry,lac__L_e --> L-Lactate -->
GPR,
Lower bound,0.4
Upper bound,1000.0


In [113]:
l_lactis.optimize().objective_value

0.801206927384408

### Obtention of sets of reactions with flux

In [161]:
# E.coli
e_coli_glc_fva = flux_variability_analysis(e_coli)

In [162]:
e_coli_glc_fva

,minimum,maximum
CYTDK2,0.000000,1.555200e-12
XPPT,0.000000,1.560529e-12
HXPRT,0.000000,0.000000e+00
NDPK5,-0.022948,-5.923262e-12
SHK3Dr,0.334240,3.342403e-01
...,...,...
MPTS,0.000000,0.000000e+00
MOCOS,0.000000,0.000000e+00
BMOGDS2,0.000000,0.000000e+00
FESD2s,0.000000,0.000000e+00


In [163]:
e_coli_glc_r = e_coli_glc_fva.loc[(e_coli_glc_fva['minimum'] != 0) & (e_coli_glc_fva['maximum'] != 0)].copy()
len(e_coli_glc_r)

430

In [164]:
# we went from 2712 to 430 reactions (~15,86%)
# extracting reactions IDs
e_coli_glc_r_ids = list(e_coli_glc_r.index)

In [165]:
e_coli_glc_r_ids[:10]

['NDPK5',
 'SHK3Dr',
 'NDPK8',
 'DHORTS',
 'OMPDC',
 'G5SD',
 'CS',
 'ICDHyr',
 'ACALD',
 'PPA']

In [166]:
# L. lactis
l_lactis_glc_fva = flux_variability_analysis(l_lactis)
l_lactis_glc_r = l_lactis_glc_fva.loc[(l_lactis_glc_fva['minimum'] != 0) & (l_lactis_glc_fva['maximum'] != 0)].copy()
l_lactis_glc_r_ids = list(l_lactis_glc_r.index)
len(l_lactis_glc_r_ids)

325

In [167]:
# P. putida
p_putida_glc_fva = flux_variability_analysis(p_putida)
p_putida_glc_r = p_putida_glc_fva.loc[(p_putida_glc_fva['minimum'] != 0) & (p_putida_glc_fva['maximum'] != 0)].copy()
p_putida_glc_r_ids = list(p_putida_glc_r.index)
len(p_putida_glc_r_ids)

596

We want to test our function in several models and two different carbon source, glucose and ribose.

In [168]:
e_coli_rib = fake_copy(e_coli)
l_lactis_rib = fake_copy(l_lactis)
p_putida_rib = fake_copy(p_putida)

In [169]:
rib_models = [e_coli_rib, l_lactis_rib, p_putida_rib]

for x in rib_models:
    x.reactions.EX_glc__D_e.lower_bound = 0.0
    x.reactions.EX_rib__D_e.lower_bound = -10.0

# setting L. lactis ribose bound to -2.12 to match Glc bound
l_lactis_rib.reactions.EX_rib__D_e.lower_bound = -2.12

In [170]:
for x in rib_models:
    print(x.optimize().objective_value)

0.6889133990312893
0.7357382398247538
0.8066668126261822


In [118]:
# l.lactis and e. coli lowers their flux; p. putida rise it

In [171]:
# fva analysis for models with ribose as cs
# E.coli
e_coli_rib_fva = flux_variability_analysis(e_coli_rib)
e_coli_rib_r = e_coli_rib_fva.loc[(e_coli_rib_fva['minimum'] != 0) & (e_coli_rib_fva['maximum'] != 0)].copy()
e_coli_rib_r_ids = list(e_coli_rib_r.index)
print(len(e_coli_rib_r_ids))

# L. lactis
l_lactis_rib_fva = flux_variability_analysis(l_lactis_rib)
l_lactis_rib_r = l_lactis_rib_fva.loc[(l_lactis_rib_fva['minimum'] != 0) & (l_lactis_rib_fva['maximum'] != 0)].copy()
l_lactis_rib_r_ids = list(l_lactis_rib_r.index)
print(len(l_lactis_rib_r_ids))

# P. putida
p_putida_rib_fva = flux_variability_analysis(p_putida_rib)
p_putida_rib_r = p_putida_rib_fva.loc[(p_putida_rib_fva['minimum'] != 0) & (p_putida_rib_fva['maximum'] != 0)].copy()
p_putida_rib_r_ids = list(p_putida_rib_r.index)
print(len(p_putida_rib_r_ids))

434
328
592


### Test 1a: removal of 1 reaction (glucose)

In [172]:
# Creating 10 copies of each model

e_coli_samples_glc = make_model_copies(e_coli)
l_lactis_samples_glc = make_model_copies(l_lactis)
p_putida_samples_glc = make_model_copies(p_putida)

Now, we will remove one random reaction of every copy. 

In [173]:
# original number of reactions
print(len(list(e_coli_samples_glc.values())[5].reactions))
print(len(list(l_lactis_samples_glc.values())[5].reactions))
print(len(list(p_putida_samples_glc.values())[5].reactions))

2712
754
2927


In [174]:
# removing one reaction from every model

remove_random_reaction(e_coli, e_coli_samples_glc, reactions_list=e_coli_glc_r_ids)
remove_random_reaction(l_lactis, l_lactis_samples_glc, reactions_list=l_lactis_glc_r_ids)
remove_random_reaction(p_putida, p_putida_samples_glc, reactions_list=p_putida_glc_r_ids)

['DHQTi', 'PMEACPE', 'ENO', 'SULR', 'OPMEACPS', 'G3PD2', 'PPBNGS', 'APRAUR', '3OAR140', 'DASYN161']
['VALTRS_1', 'METt2r', 'BTMAT1', 'GLYTRS_1', 'PFK', 'UDCPDPS', 'EX_thm_e', 'THRD_L', 'OCDMAT8', 'LTAS_LLA']
['PGSA160', 'PSD181', 'PGCD', 'NADH16pp', 'H2Otpp', 'PACCOAL', 'OCBT', 'CFAS160E', '3OAS100', 'THRA2']


{'iJN1463_1': <Model iJN1463 at 0x7f3000770130>,
 'iJN1463_2': <Model iJN1463 at 0x7f306bbdbee0>,
 'iJN1463_3': <Model iJN1463 at 0x7f301a3dda00>,
 'iJN1463_4': <Model iJN1463 at 0x7f30115227c0>,
 'iJN1463_5': <Model iJN1463 at 0x7f300b6556a0>,
 'iJN1463_6': <Model iJN1463 at 0x7f30131d6220>,
 'iJN1463_7': <Model iJN1463 at 0x7f2ffb7ef970>,
 'iJN1463_8': <Model iJN1463 at 0x7f3041fde280>,
 'iJN1463_9': <Model iJN1463 at 0x7f304130de20>,
 'iJN1463_10': <Model iJN1463 at 0x7f2fe8ffb880>}

In [175]:
print(len(list(e_coli_samples_glc.values())[5].reactions))
print(len(list(l_lactis_samples_glc.values())[5].reactions))
print(len(list(p_putida_samples_glc.values())[5].reactions))

2711
753
2926


#### E. coli

In [176]:
# checking functionality of the models
for i in range(len(e_coli_samples_glc)):
    print(list(e_coli_samples_glc.keys())[i])
    print(list(e_coli_samples_glc.values())[i].optimize().objective_value)

iML1515_1
0.0
iML1515_2
0.0
iML1515_3
0.8035669172773582
iML1515_4
0.0
iML1515_5
0.0
iML1515_6
0.8766533613913154
iML1515_7
0.0
iML1515_8
0.0
iML1515_9
0.0
iML1515_10
0.0


In [177]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples_glc:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples_glc[x], e_coli, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
[<Reaction DHQTi at 0x7f3025d73a30>]
iML1515_2
iML1515_3
[]
iML1515_4
[<Reaction SULR at 0x7f3055fce2e0>]
iML1515_5
iML1515_6
[]
iML1515_7
iML1515_8
iML1515_9
[<Reaction 3OAR140 at 0x7f2fdb160310>]
iML1515_10
[<Reaction DASYN161 at 0x7f2fc9cb6bb0>]
2min 40s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [178]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples_glc[x], [e_coli], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('DHQTi', ['b1693'])]}
iML1515_2
{}
iML1515_3
{'iML1515': []}
iML1515_4
{'iML1515': [('SULR', ['b2763', 'b2764'])]}
iML1515_5
{}
iML1515_6
{'iML1515': []}
iML1515_7
{'iML1515': [('PPBNGS', ['b0369'])]}
iML1515_8
{'iML1515': [('APRAUR', ['b0414'])]}
iML1515_9
{'iML1515': [('3OAR140', ['b1093'])]}
iML1515_10
{'iML1515': [('DASYN161', ['b0175'])]}
3min 17s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### L. lactis

In [182]:
for i in range(len(l_lactis_samples_glc)):
    print(list(l_lactis_samples_glc.keys())[i])
    print(list(l_lactis_samples_glc.values())[i].optimize().objective_value)

iNF517_1
0.0
iNF517_2
0.8003873811152523
iNF517_3
0.0
iNF517_4
0.0
iNF517_5
0.0014450588620622544
iNF517_6
0.0
iNF517_7
0.0
iNF517_8
0.7815898078090185
iNF517_9
0.0
iNF517_10
0.0


In [183]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples_glc:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples_glc[x], l_lactis, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
[<Reaction VALTRS_1 at 0x7f2fb4ff1100>]
iNF517_2
[]
iNF517_3
[<Reaction BTMAT1 at 0x7f2fda63dbb0>]
iNF517_4
[<Reaction GLYTRS_1 at 0x7f3023c89e20>]
iNF517_5
[<Reaction PFK at 0x7f30185f3610>]
iNF517_6
iNF517_7
iNF517_8
[]
iNF517_9
iNF517_10
47.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [184]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples_glc[x], [l_lactis], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{'iNF517': [('VALTRS_1', ['LLMG_RS12325'])]}
iNF517_2
{'iNF517': [('Transport reaction: METt2r', [])]}
iNF517_3
{'iNF517': [('BTMAT1', ['LLMG_RS02820'])]}
iNF517_4
{'iNF517': [('GLYTRS_1', ['LLMG_RS07455', 'LLMG_RS07450'])]}
iNF517_5
{'iNF517': [('PFK', ['LLMG_RS05680'])]}
iNF517_6
{}
iNF517_7
{'iNF517': [('EX_thm_e', 'Exchange reaction')]}
iNF517_8
{'iNF517': []}
iNF517_9
{'iNF517': [('OCDMAT8', ['LLMG_RS02820'])]}
iNF517_10
{'iNF517': [('LTAS_LLA', [])]}
58.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [266]:
1/8

0.125

#### P. putida

In [185]:
for i in range(len(p_putida_samples_glc)):
    print(list(p_putida_samples_glc.keys())[i])
    print(list(p_putida_samples_glc.values())[i].optimize().objective_value)

iJN1463_1
0.0
iJN1463_2
0.0
iJN1463_3
0.0
iJN1463_4
0.5247973972311358
iJN1463_5
0.08796688835043205
iJN1463_6
0.5861175448479808
iJN1463_7
0.0
iJN1463_8
0.0
iJN1463_9
0.0
iJN1463_10
0.0


In [186]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples_glc:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples_glc[x], p_putida, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
iJN1463_2
iJN1463_3
iJN1463_4
[]
iJN1463_5
[]
iJN1463_6
[]
iJN1463_7
iJN1463_8
iJN1463_9
iJN1463_10
2min 58s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [187]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples_glc[x], [p_putida], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{}
iJN1463_2
{}
iJN1463_3
{'iJN1463': [('PGCD', ['PP_5155'])]}
iJN1463_4
{'iJN1463': [('Transport reaction: NADH16pp', ['PP_4123', 'PP_4121', 'PP_4124', 'PP_4126', 'PP_4125', 'PP_4120', 'PP_4119', 'PP_4131', 'PP_4122', 'PP_4130', 'PP_4127', 'PP_4128', 'PP_4129'])]}
iJN1463_5
{'iJN1463': [('Transport reaction: H2Otpp', ['PP_s0001', 'PP_4282'])]}
iJN1463_6
{'iJN1463': []}
iJN1463_7
{'iJN1463': [('OCBT', ['PP_1079', 'PP_1000'])]}
iJN1463_8
{}
iJN1463_9
{'iJN1463': [('3OAS100', ['PP_4175', 'PP_3303', 'PP_1916'])]}
iJN1463_10
{}
3min 26s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Test 1b: removal of 1 reaction (ribose)

In [188]:
e_coli_samples_rib = make_model_copies(e_coli_rib)
l_lactis_samples_rib = make_model_copies(l_lactis_rib)
p_putida_samples_rib = make_model_copies(p_putida_rib)

In [189]:
remove_random_reaction(e_coli_rib, e_coli_samples_rib, reactions_list=e_coli_rib_r_ids)
remove_random_reaction(l_lactis_rib, l_lactis_samples_rib, reactions_list=l_lactis_rib_r_ids)
remove_random_reaction(p_putida_rib, p_putida_samples_rib, reactions_list=p_putida_rib_r_ids)

['DHAD1', 'RBK', 'OMCDC', 'PMDPHT', 'ASPtpp', 'DALAt2pp', 'PSP_L', 'HACD4', 'GLUR', 'ACOAD5f']
['NAPRT', 'MCMAT4', 'DTMPK', 'PROTRS_1', 'CTPS1', 'ABUTt2r', 'GTPCI', 'EX_asp__L_e', 'EX_ala__L_e', 'ARGORNt3']
['PYOVDM1', 'EX_mn2_e', 'HPYRRx', 'NNDMBRT', 'NI2tpp', 'MCTP2App', 'H2CO3D2', 'BMOGDS2', 'ADCYRS', 'MTHFD']


{'iJN1463_1': <Model iJN1463 at 0x7f2fb0c5eb80>,
 'iJN1463_2': <Model iJN1463 at 0x7f302c843880>,
 'iJN1463_3': <Model iJN1463 at 0x7f2fba614d30>,
 'iJN1463_4': <Model iJN1463 at 0x7f2facaba310>,
 'iJN1463_5': <Model iJN1463 at 0x7f2f9c2841c0>,
 'iJN1463_6': <Model iJN1463 at 0x7f2f92baa100>,
 'iJN1463_7': <Model iJN1463 at 0x7f2fbd6b7e80>,
 'iJN1463_8': <Model iJN1463 at 0x7f2fb5514250>,
 'iJN1463_9': <Model iJN1463 at 0x7f2fa3a54370>,
 'iJN1463_10': <Model iJN1463 at 0x7f2f97e08b20>}

#### E. coli

In [190]:
# checking functionality of the models
for i in range(len(e_coli_samples_rib)):
    print(list(e_coli_samples_rib.keys())[i])
    print(list(e_coli_samples_rib.values())[i].optimize().objective_value)

iML1515_1
0.0
iML1515_2


/home/fco/miniconda3/envs/TFM/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


None
iML1515_3
0.0
iML1515_4
0.0
iML1515_5
0.6889133990312898
iML1515_6
0.6885085709876102
iML1515_7
0.6750829044838003
iML1515_8
0.6848306747602794
iML1515_9
0.0
iML1515_10
0.6854889702091407


In [191]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples_rib[x], e_coli_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
[<Reaction DHAD1 at 0x7f2f86a11d30>]
iML1515_2
[<Reaction RBK at 0x7f301841f2e0>]
iML1515_3
[<Reaction OMCDC at 0x7f2fd857b3a0>]
iML1515_4
iML1515_5
[]
iML1515_6
[]
iML1515_7
[]
iML1515_8
[]
iML1515_9
[<Reaction GLUR at 0x7f2f70f12760>]
iML1515_10
[]
2min 43s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [192]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples_rib[x], [e_coli_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('DHAD1', ['b3771'])]}
iML1515_2
{'iML1515': [('RBK', ['b3752'])]}
iML1515_3
{'iML1515': [('OMCDC', ['s0001'])]}
iML1515_4
{'iML1515': [('PMDPHT', ['b0844', 'b3812'])]}
iML1515_5
{'iML1515': [('Transport reaction: ASPtpp', ['b1206'])]}
iML1515_6
{'iML1515': [('Transport reaction: DALAt2pp', ['b4208'])]}
iML1515_7
{'iML1515': []}
iML1515_8
{'iML1515': []}
iML1515_9
{'iML1515': [('GLUR', ['b3967'])]}
iML1515_10
{'iML1515': []}
3min 22s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [267]:
2/7

0.2857142857142857

#### L. lactis

In [193]:
# checking functionality of the models
for i in range(len(l_lactis_samples_rib)):
    print(list(l_lactis_samples_rib.keys())[i])
    print(list(l_lactis_samples_rib.values())[i].optimize().objective_value)

iNF517_1
0.0
iNF517_2
0.0
iNF517_3
0.0
iNF517_4
0.0
iNF517_5
0.7051284187920162
iNF517_6
0.7313938106350848
iNF517_7
0.0
iNF517_8
0.7300471172912578
iNF517_9
0.7324555516963561
iNF517_10
0.7357280183015322


In [194]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples_rib[x], l_lactis_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
iNF517_2
[<Reaction MCMAT4 at 0x7f2fba43d2e0>]
iNF517_3
[<Reaction DTMPK at 0x7f2f890bcd00>]
iNF517_4
[<Reaction PROTRS_1 at 0x7f2f72bf3490>]
iNF517_5
[]
iNF517_6
[]
iNF517_7
iNF517_8
[]
iNF517_9
[]
iNF517_10
[]
47.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [195]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples_rib[x], [l_lactis_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{'iNF517': [('NAPRT', ['LLMG_RS07425'])]}
iNF517_2
{'iNF517': [('MCMAT4', ['LLMG_RS08945'])]}
iNF517_3
{'iNF517': [('DTMPK', ['LLMG_RS02125'])]}
iNF517_4
{'iNF517': [('PROTRS_1', ['LLMG_RS12095'])]}
iNF517_5
{'iNF517': []}
iNF517_6
{'iNF517': [('Transport reaction: ABUTt2r', [])]}
iNF517_7
{}
iNF517_8
{'iNF517': [('EX_asp__L_e', 'Exchange reaction')]}
iNF517_9
{'iNF517': [('EX_ala__L_e', 'Exchange reaction')]}
iNF517_10
{'iNF517': [('Transport reaction: ARGORNt3', ['LLMG_RS11585', 'LLMG_RS11565'])]}
58.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### P. putida

In [196]:
# checking functionality of the models
for i in range(len(p_putida_samples_rib)):
    print(list(p_putida_samples_rib.keys())[i])
    print(list(p_putida_samples_rib.values())[i].optimize().objective_value)

iJN1463_1
0.0
iJN1463_2
0.0
iJN1463_3
0.8066668126261828
iJN1463_4
0.0
iJN1463_5
0.8066565289528399
iJN1463_6
0.0
iJN1463_7
0.8066668126261851
iJN1463_8
0.0
iJN1463_9
0.0
iJN1463_10
0.780624801337949


In [197]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples_rib[x], p_putida_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
iJN1463_2
iJN1463_3
[]
iJN1463_4
iJN1463_5
[]
iJN1463_6
iJN1463_7
[]
iJN1463_8
iJN1463_9
iJN1463_10
[]
3min 1s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [198]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples_rib[x], [p_putida_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{}
iJN1463_2
{'iJN1463': [('EX_mn2_e', 'Exchange reaction')]}
iJN1463_3
{'iJN1463': []}
iJN1463_4
{}
iJN1463_5
{'iJN1463': [('Transport reaction: NI2tpp', ['PP_1843'])]}
iJN1463_6
{}
iJN1463_7
{'iJN1463': []}
iJN1463_8
{}
iJN1463_9
{}
iJN1463_10
{'iJN1463': []}
3min 12s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Test 2a: removal of 2 reactions (glucose)

In [199]:
e_coli_samples_glc = make_model_copies(e_coli)
l_lactis_samples_glc = make_model_copies(l_lactis)
p_putida_samples_glc = make_model_copies(p_putida)

In [200]:
remove_random_reaction(e_coli, e_coli_samples_glc, reactions_list=e_coli_glc_r_ids, n=2)
remove_random_reaction(l_lactis, l_lactis_samples_glc, reactions_list=l_lactis_glc_r_ids, n=2)
remove_random_reaction(p_putida, p_putida_samples_glc, reactions_list=p_putida_glc_r_ids, n=2)

['ACACT4r', 'IGPDH', 'I4FE4SR', 'UPPDC1', 'NNDPR', 'MEOHtex', 'UAGPT3', 'IPPMIa', 'OMCDC', 'ACACT1r', 'UAGDP', 'EX_ni2_e', 'ENO', 'IMPD', 'PRFGS', 'GLUDy', 'HSTPT', 'A5PISO', 'FUM', 'CLtex']
['ETOHt', '4ABZt', 'PGGT2', 'FABM1', 'GLUR', 'EX_thr__L_e', 'FABM2', 'EX_h2o_e', 'ILEt2r_copy2', 'ARGORNt3', 'DEMAT4', 'aratyr3', 'araphe1', 'UAGDP', 'UAGCVT', 'EX_ser__L_e', 'PSCVT', 'NDPK3', 'VALTA', 'UAGPT1']
['DDPA', 'G5SD_copy2', 'LPSGTR1_kt', 'FEROpp', 'NNDMBRT', 'ASPK', 'GHMT2r', 'DHDPS', 'EX_so4_e', 'GLUPRT', '3OAR160', 'MOADSUx', 'UAMAS', 'PGM', 'VALTA', '3HAACOAT141', 'DMATT', 'ALATRS', 'ACONTb', '3HAD80']


{'iJN1463_1': <Model iJN1463 at 0x7f2f363e9580>,
 'iJN1463_2': <Model iJN1463 at 0x7f2f75efa520>,
 'iJN1463_3': <Model iJN1463 at 0x7f2fa3c21e20>,
 'iJN1463_4': <Model iJN1463 at 0x7f2f806392e0>,
 'iJN1463_5': <Model iJN1463 at 0x7f2f5ec83e20>,
 'iJN1463_6': <Model iJN1463 at 0x7f2f81db6160>,
 'iJN1463_7': <Model iJN1463 at 0x7f2fe8fabfd0>,
 'iJN1463_8': <Model iJN1463 at 0x7f2f7ffa1100>,
 'iJN1463_9': <Model iJN1463 at 0x7f3009da5e50>,
 'iJN1463_10': <Model iJN1463 at 0x7f2fdb2ec070>}

#### E. coli

In [201]:
# checking functionality of the models
for i in range(len(e_coli_samples_glc)):
    print(list(e_coli_samples_glc.keys())[i])
    print(list(e_coli_samples_glc.values())[i].optimize().objective_value)

iML1515_1
0.0
iML1515_2
0.0
iML1515_3
0.0
iML1515_4
0.0
iML1515_5
0.0
iML1515_6
0.0
iML1515_7
0.8020515134629888
iML1515_8
0.0
iML1515_9
0.0
iML1515_10
0.0


In [202]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples_glc:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples_glc[x], e_coli, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
[<Reaction IGPDH at 0x7f2fcb8a4af0>]
iML1515_2
iML1515_3
iML1515_4
[<Reaction IPPMIa at 0x7f2f8b7b3a90>, <Reaction UAGPT3 at 0x7f2f8b5e8d30>]
iML1515_5
[<Reaction OMCDC at 0x7f2f39e989a0>]
iML1515_6
iML1515_7
[]
iML1515_8
[<Reaction PRFGS at 0x7f2f1a8d6a90>]
iML1515_9
[<Reaction A5PISO at 0x7f2f144352b0>, <Reaction HSTPT at 0x7f2f142e9970>]
iML1515_10
2min 46s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [203]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples_glc[x], [e_coli], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('IGPDH', ['b2022'])]}
iML1515_2
{'iML1515': [('UPPDC1', ['b3997'])]}
iML1515_3
{'iML1515': [('Transport reaction: MEOHtex', ['b1377', 'b2215', 'b0241', 'b0929']), ('NNDPR', ['b0109'])]}
iML1515_4
{'iML1515': [('IPPMIa', ['b0071', 'b0072']), ('UAGPT3', ['b0090'])]}
iML1515_5
{'iML1515': [('OMCDC', ['s0001'])]}
iML1515_6
{'iML1515': [('EX_ni2_e', 'Exchange reaction'), ('UAGDP', ['b3730'])]}
iML1515_7
{'iML1515': []}
iML1515_8
{'iML1515': [('PRFGS', ['b2557'])]}
iML1515_9
{'iML1515': [('A5PISO', ['b3197', 'b2708']), ('HSTPT', ['b2021'])]}
iML1515_10
{'iML1515': [('Transport reaction: CLtex', ['b1377', 'b2215', 'b0241', 'b0929'])]}
3min 36s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### L. lactis

In [204]:
for i in range(len(l_lactis_samples_glc)):
    print(list(l_lactis_samples_glc.keys())[i])
    print(list(l_lactis_samples_glc.values())[i].optimize().objective_value)

iNF517_1
None
iNF517_2
0.0
iNF517_3
0.0
iNF517_4
0.0
iNF517_5
None
iNF517_6
0.0
iNF517_7
0.0
iNF517_8
0.0
iNF517_9
0.0
iNF517_10
None


In [205]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples_glc:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples_glc[x], l_lactis, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
[<Reaction ETOHt at 0x7f2ef4a95ac0>]
iNF517_2
iNF517_3
[<Reaction GLUR at 0x7f2f98f1f790>, <Reaction EX_thr__L_e at 0x7f2fbf3e3b50>]
iNF517_4
[<Reaction FABM2 at 0x7f2f449240d0>]
iNF517_5
[<Reaction ILEt2r_copy2 at 0x7f2fd3dde940>]
iNF517_6
[<Reaction DEMAT4 at 0x7f2f1cb69d60>]
iNF517_7
[<Reaction UAGDP at 0x7f2f22d3d4c0>]
iNF517_8
[<Reaction EX_ser__L_e at 0x7f2f2c7c90d0>, <Reaction UAGCVT at 0x7f2f2295dd30>]
iNF517_9
[<Reaction PSCVT at 0x7f2fa14fb910>]
iNF517_10
[<Reaction UAGPT1 at 0x7f2f0ce54d00>]
47.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [206]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples_glc[x], [l_lactis], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{'iNF517': [('Transport reaction: ETOHt', []), ('Transport reaction: 4ABZt', [])]}
iNF517_2
{'iNF517': [('PGGT2', ['LLMG_RS01845', 'LLMG_RS08445', 'LLMG_RS11990', 'LLMG_RS02600']), ('FABM1', [])]}
iNF517_3
{'iNF517': [('EX_thr__L_e', 'Exchange reaction'), ('GLUR', ['LLMG_RS06010'])]}
iNF517_4
{'iNF517': [('EX_h2o_e', 'Exchange reaction'), ('FABM2', [])]}
iNF517_5
{'iNF517': [('Transport reaction: ARGORNt3', ['LLMG_RS11585', 'LLMG_RS11565']), ('Transport reaction: ILEt2r_copy2', ['LLMG_RS03390'])]}
iNF517_6
{'iNF517': [('DEMAT4', ['LLMG_RS02820'])]}
iNF517_7
{'iNF517': [('UAGDP', ['LLMG_RS10360'])]}
iNF517_8
{'iNF517': [('EX_ser__L_e', 'Exchange reaction'), ('UAGCVT', ['LLMG_RS02630', 'LLMG_RS01685'])]}
iNF517_9
{'iNF517': [('PSCVT', ['LLMG_RS09640'])]}
iNF517_10
{'iNF517': [('UAGPT1', ['LLMG_RS04660'])]}
1min 2s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### P. putida

In [207]:
for i in range(len(p_putida_samples_glc)):
    print(list(p_putida_samples_glc.keys())[i])
    print(list(p_putida_samples_glc.values())[i].optimize().objective_value)

iJN1463_1
0.0
iJN1463_2
0.0
iJN1463_3
0.0
iJN1463_4
0.0
iJN1463_5
0.0
iJN1463_6
0.0
iJN1463_7
0.0
iJN1463_8
0.5860448916096798
iJN1463_9
0.0
iJN1463_10
0.0


In [208]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples_glc:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples_glc[x], p_putida, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
iJN1463_2
iJN1463_3
iJN1463_4
iJN1463_5
iJN1463_6
iJN1463_7
iJN1463_8
[]
iJN1463_9
iJN1463_10
3min 4s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [209]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples_glc[x], [p_putida], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{'iJN1463': [('DDPA', ['PP_3080', 'PP_1866', 'PP_2324'])]}
iJN1463_2
{}
iJN1463_3
{}
iJN1463_4
{'iJN1463': [('DHDPS', ['PP_1237'])]}
iJN1463_5
{'iJN1463': [('EX_so4_e', 'Exchange reaction'), ('GLUPRT', ['PP_2000'])]}
iJN1463_6
{}
iJN1463_7
{'iJN1463': [('UAMAS', ['PP_0547', 'PP_1338'])]}
iJN1463_8
{'iJN1463': []}
iJN1463_9
{}
iJN1463_10
{'iJN1463': [('3HAD80', ['PP_1602', 'PP_4174'])]}
3min 30s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Test 2b: removal of 2 reactions (ribose)

In [210]:
e_coli_samples_rib = make_model_copies(e_coli_rib)
l_lactis_samples_rib = make_model_copies(l_lactis_rib)
p_putida_samples_rib = make_model_copies(p_putida_rib)

In [211]:
remove_random_reaction(e_coli_rib, e_coli_samples_rib, reactions_list=e_coli_rib_r_ids, n=2)
remove_random_reaction(l_lactis_rib, l_lactis_samples_rib, reactions_list=l_lactis_rib_r_ids, n=2)
remove_random_reaction(p_putida_rib, p_putida_samples_rib, reactions_list=p_putida_rib_r_ids, n=2)

['ALAR', 'DHDPRy', 'EX_cu2_e', 'VPAMTr', 'EX_rib__D_e', 'MTHFD', 'COBALT2tpp', 'UPPDC1', 'GLU5K', 'UPP3S', 'FADRx', 'METAT', 'AIRC3', 'MDH', 'ASAD', 'KARA1', 'DHORTS', 'EX_h_e', 'PSERT', 'AICART']
['HPPK', 'DHFR', 'DHNPA_1', 'GHMT2r', 'TYRTA_copy2', 'UGLDDS1', 'LPGS_LLA', 'GUAPRT', 'PRPPS', 'DHAD1', 'PHEt2r', 'BIOMASS_LLA', 'PRAIi', 'DHQS', 'UDCPDP', 'FOLD3', 'EX_ile__L_e', 'CYTK2', 'NDPK7', 'EX_nh4_e']
['I2FE2ST', 'LPSALAT', 'PG181abcpp', 'FEROpp', 'MECDPDH5', '3HAD161', 'EGMEACPR', 'PSD140', 'INDOLEtex', 'LPSGTR2_kt', 'DXPS', 'PIt2rpp', 'AGPR', 'KAT26', 'RPE', 'FMNRx2_copy1', 'FRDO6r', 'AMAOTr', 'DABAAT2', 'RZ5PP']


{'iJN1463_1': <Model iJN1463 at 0x7f2f75cbe610>,
 'iJN1463_2': <Model iJN1463 at 0x7f2fd79887f0>,
 'iJN1463_3': <Model iJN1463 at 0x7f3034a1cf70>,
 'iJN1463_4': <Model iJN1463 at 0x7f2ff6825c70>,
 'iJN1463_5': <Model iJN1463 at 0x7f2fe77dcee0>,
 'iJN1463_6': <Model iJN1463 at 0x7f3036e75220>,
 'iJN1463_7': <Model iJN1463 at 0x7f2f687bd040>,
 'iJN1463_8': <Model iJN1463 at 0x7f2ed68ee040>,
 'iJN1463_9': <Model iJN1463 at 0x7f3101c7cf40>,
 'iJN1463_10': <Model iJN1463 at 0x7f2f1f5dd2b0>}

#### E. coli

In [212]:
# checking functionality of the models
for i in range(len(e_coli_samples_rib)):
    print(list(e_coli_samples_rib.keys())[i])
    print(list(e_coli_samples_rib.values())[i].optimize().objective_value)

iML1515_1
0.0
iML1515_2
0.0
iML1515_3
None
iML1515_4
0.0
iML1515_5
0.0
iML1515_6
0.0
iML1515_7
0.0
iML1515_8
0.0
iML1515_9
0.0
iML1515_10
0.0


In [213]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples_rib[x], e_coli_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
[<Reaction ALAR at 0x7f2f29a035b0>, <Reaction DHDPRy at 0x7f2ecc2c40a0>]
iML1515_2
iML1515_3
[<Reaction EX_rib__D_e at 0x7f2f1f221220>]
iML1515_4
iML1515_5
iML1515_6
iML1515_7
[<Reaction AIRC3 at 0x7f2eebf33f70>]
iML1515_8
[<Reaction KARA1 at 0x7f2ed1028880>, <Reaction ASAD at 0x7f2ed1005760>]
iML1515_9
[<Reaction DHORTS at 0x7f2ec3067970>]
iML1515_10
[<Reaction AICART at 0x7f2ebca2c8e0>]
2min 41s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [214]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples_rib[x], [e_coli_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('ALAR', ['b4053', 'b1190']), ('DHDPRy', ['b0031'])]}
iML1515_2
{'iML1515': [('EX_cu2_e', 'Exchange reaction')]}
iML1515_3
{'iML1515': [('EX_rib__D_e', 'Exchange reaction')]}
iML1515_4
{'iML1515': [('Transport reaction: COBALT2tpp', ['b3040', 'b3816']), ('UPPDC1', ['b3997'])]}
iML1515_5
{'iML1515': [('UPP3S', ['b3804']), ('GLU5K', ['b0242'])]}
iML1515_6
{'iML1515': [('METAT', ['b2942'])]}
iML1515_7
{'iML1515': [('AIRC3', ['b0523'])]}
iML1515_8
{'iML1515': [('KARA1', ['b3774']), ('ASAD', ['b3433'])]}
iML1515_9
{'iML1515': [('EX_h_e', 'Exchange reaction'), ('DHORTS', ['b1062'])]}
iML1515_10
{'iML1515': [('AICART', ['b4006'])]}
3min 39s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### L. lactis

In [215]:
# checking functionality of the models
for i in range(len(l_lactis_samples_rib)):
    print(list(l_lactis_samples_rib.keys())[i])
    print(list(l_lactis_samples_rib.values())[i].optimize().objective_value)

iNF517_1
0.0
iNF517_2
0.0
iNF517_3
0.0
iNF517_4
0.0
iNF517_5
0.0
iNF517_6
0.0
iNF517_7
0.0
iNF517_8
0.0
iNF517_9
0.0
iNF517_10
0.1606519698143803


In [216]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples_rib[x], l_lactis_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
iNF517_2
iNF517_3
[<Reaction UGLDDS1 at 0x7f2f19261af0>]
iNF517_4
iNF517_5
[<Reaction PRPPS at 0x7f2edb815880>]
iNF517_6
iNF517_7
[<Reaction DHQS at 0x7f2ea1db1040>, <Reaction PRAIi at 0x7f2ebea9d040>]
iNF517_8
iNF517_9
[<Reaction CYTK2 at 0x7f2ebba65400>, <Reaction EX_ile__L_e at 0x7f2ed9bb4e20>]
iNF517_10
[]
47.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [217]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples_rib[x], [l_lactis_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{}
iNF517_2
{}
iNF517_3
{'iNF517': [('UGLDDS1', ['LLMG_RS01860'])]}
iNF517_4
{'iNF517': [('LPGS_LLA', [])]}
iNF517_5
{'iNF517': [('PRPPS', ['LLMG_RS08760', 'LLMG_RS10225'])]}
iNF517_6
{}
iNF517_7
{'iNF517': [('DHQS', ['LLMG_RS09700']), ('PRAIi', ['LLMG_RS05270'])]}
iNF517_8
{}
iNF517_9
{'iNF517': [('EX_ile__L_e', 'Exchange reaction'), ('CYTK2', ['LLMG_RS03780'])]}
iNF517_10
{'iNF517': [('EX_nh4_e', 'Exchange reaction')]}
57.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### P.putida

In [218]:
# checking functionality of the models
for i in range(len(p_putida_samples_rib)):
    print(list(p_putida_samples_rib.keys())[i])
    print(list(p_putida_samples_rib.values())[i].optimize().objective_value)

iJN1463_1
0.0
iJN1463_2
0.0
iJN1463_3
0.0
iJN1463_4
0.0
iJN1463_5
0.0
iJN1463_6
0.0
iJN1463_7
0.0
iJN1463_8
0.5289571570684432
iJN1463_9
0.0
iJN1463_10
0.0


In [219]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples_rib[x], p_putida_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
iJN1463_2
iJN1463_3
iJN1463_4
iJN1463_5
iJN1463_6
iJN1463_7
iJN1463_8
[]
iJN1463_9
iJN1463_10
2min 59s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [220]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples_rib[x], [p_putida_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{}
iJN1463_2
{}
iJN1463_3
{}
iJN1463_4
{}
iJN1463_5
{}
iJN1463_6
{}
iJN1463_7
{'iJN1463': [('AGPR', ['PP_0432', 'PP_3633'])]}
iJN1463_8
{'iJN1463': []}
iJN1463_9
{}
iJN1463_10
{}
3min 8s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Test 3a: removal of 5 reactions (glucose)

In [221]:
e_coli_samples_glc = make_model_copies(e_coli)
l_lactis_samples_glc = make_model_copies(l_lactis)
p_putida_samples_glc = make_model_copies(p_putida)

In [222]:
remove_random_reaction(e_coli, e_coli_samples_glc, reactions_list=e_coli_glc_r_ids, n=5)
remove_random_reaction(l_lactis, l_lactis_samples_glc, reactions_list=l_lactis_glc_r_ids, n=5)
remove_random_reaction(p_putida, p_putida_samples_glc, reactions_list=p_putida_glc_r_ids, n=5)

['ADCL', 'IGPS', 'ASNS2', 'Kt2pp', 'DNTPPA', 'HACD5', 'ACOAD3f', 'AHCYSNS', 'PGK', 'NADS1', 'CU2tex', 'ACALD', 'DAPE', 'EX_meoh_e', 'UAGPT3', 'COBALT2tpp', 'SHK3Dr', 'MDH', 'MOBDtex', 'DHORTS', 'SULR', 'DASYN161', 'MG2tex', 'AI2K', 'EX_so4_e', 'OPMEACPD', 'CYTK1', 'CPPPGO', 'THRS', 'CLt3_2pp', 'MECDPDH5', 'IMPC', 'USHD', 'I2FE2ST', 'PPM', 'EX_mobd_e', 'DNMPPA', 'ECOAH1', 'GAPD', 'MEOHtex', 'OPMEACPR', 'HCO3E', 'AIRC2', 'PGM', 'TKT1', 'ECOAH5', 'ANS', 'MCTP1App', 'DHFR', 'U23GAAT']
['EX_thr__L_e', 'HDDR5', 'PROt3', 'ADEt2', 'HDER4', 'ACTD', 'HDDHL5', 'ALATA_L', 'HISt2r', 'KARA2', 'DDMAT5', 'DPMVD', 'UAGCVT', 'MCMAT2', '2MBALDt', 'EX_cys__L_e', 'TYRTA_copy1', 'DALTAL_LLA', 'CTPS1', 'HOCR3', 'NDPK1', 'UGLDDS1', 'HHYHL2', 'ABUTt2r', 'PGMT', 'HBUHL1', 'ACACT1r', 'aratry2', 'ILEt2r_copy2', 'EX_met__L_e', 'araphe2', 'NADS1', 'FORt2', 'PFL', 'UAMAGS', 'THRA', 'EX_pro__L_e', 'G6PDH2r', 'GLUDC', 'IGPS', 'EX_ac_e', 'GUAPRT', 'RNAS_LLA', 'unkFol', 'ASNTRS_1', 'TRPS1', 'HTDHL6', 'FBA', 'GLNTRS2', '

{'iJN1463_1': <Model iJN1463 at 0x7f2e803554c0>,
 'iJN1463_2': <Model iJN1463 at 0x7f2e920b5a60>,
 'iJN1463_3': <Model iJN1463 at 0x7f2eb5832a30>,
 'iJN1463_4': <Model iJN1463 at 0x7f2e9daae310>,
 'iJN1463_5': <Model iJN1463 at 0x7f2e95244220>,
 'iJN1463_6': <Model iJN1463 at 0x7f2ef0804310>,
 'iJN1463_7': <Model iJN1463 at 0x7f2ea9eeaf10>,
 'iJN1463_8': <Model iJN1463 at 0x7f2ea65f27c0>,
 'iJN1463_9': <Model iJN1463 at 0x7f2ebf1d3e20>,
 'iJN1463_10': <Model iJN1463 at 0x7f2e94d3f700>}

#### E. coli

In [223]:
for i in range(len(e_coli_samples_glc)):
    print(list(e_coli_samples_glc.keys())[i])
    print(list(e_coli_samples_glc.values())[i].optimize().objective_value)

iML1515_1
0.0
iML1515_2
3.1544694820851962e-12
iML1515_3
0.0
iML1515_4
0.0
iML1515_5
0.0
iML1515_6
0.0
iML1515_7
0.0
iML1515_8
0.0
iML1515_9
0.0
iML1515_10
0.0


In [224]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples_glc:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples_glc[x], e_coli, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
iML1515_2
iML1515_3
iML1515_4
iML1515_5
iML1515_6
iML1515_7
iML1515_8
iML1515_9
iML1515_10
[<Reaction ANS at 0x7f2e6b35cdf0>, <Reaction U23GAAT at 0x7f2e6b2e0c70>, <Reaction DHFR at 0x7f2e6b1ab700>, <Reaction MCTP1App at 0x7f2e6b1328e0>]
2min 48s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples_glc[x], [e_coli], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

#### L. lactis

In [226]:
for i in range(len(l_lactis_samples_glc)):
    print(list(l_lactis_samples_glc.keys())[i])
    print(list(l_lactis_samples_glc.values())[i].optimize().objective_value)

iNF517_1
None
iNF517_2
0.0
iNF517_3
0.0
iNF517_4
0.0
iNF517_5
0.0
iNF517_6
None
iNF517_7
None
iNF517_8
0.0
iNF517_9
0.0
iNF517_10
0.0


In [227]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples_glc:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples_glc[x], l_lactis, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
[<Reaction HDDR5 at 0x7f2e534389a0>, <Reaction HDER4 at 0x7f2e53438be0>, <Reaction PROt3 at 0x7f2e53519430>, <Reaction EX_thr__L_e at 0x7f2e53519d00>]
iNF517_2
[<Reaction HDDHL5 at 0x7f2ecba72e20>]
iNF517_3
iNF517_4
iNF517_5
[<Reaction NDPK1 at 0x7f2e7516ed60>, <Reaction PGMT at 0x7f2e7516e250>, <Reaction UGLDDS1 at 0x7f2e7a57f3a0>, <Reaction HHYHL2 at 0x7f2e7a57f880>]
iNF517_6
iNF517_7
iNF517_8
[<Reaction IGPS at 0x7f2eadecea30>, <Reaction EX_pro__L_e at 0x7f2fb8ca07f0>]
iNF517_9
iNF517_10
46.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [228]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples_glc[x], [l_lactis], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{'iNF517': [('EX_thr__L_e', 'Exchange reaction'), ('Transport reaction: ADEt2', []), ('Transport reaction: PROt3', []), ('HDDR5', ['LLMG_RS08950']), ('HDER4', ['LLMG_RS08950'])]}
iNF517_2
{'iNF517': [('Transport reaction: HISt2r', []), ('HDDHL5', ['LLMG_RS08935', 'LLMG_RS02815'])]}
iNF517_3
{'iNF517': [('Transport reaction: 2MBALDt', []), ('DDMAT5', ['LLMG_RS02820']), ('DPMVD', ['LLMG_RS02175']), ('MCMAT2', ['LLMG_RS08945']), ('UAGCVT', ['LLMG_RS02630', 'LLMG_RS01685'])]}
iNF517_4
{'iNF517': [('EX_cys__L_e', 'Exchange reaction'), ('DALTAL_LLA', ['LLMG_RS06175', 'LLMG_RS06165', 'LLMG_RS06170', 'LLMG_RS06160']), ('HOCR3', ['LLMG_RS08950'])]}
iNF517_5
{'iNF517': [('Transport reaction: ABUTt2r', []), ('NDPK1', ['LLMG_RS05685']), ('PGMT', ['LLMG_RS02290']), ('UGLDDS1', ['LLMG_RS01860']), ('HHYHL2', ['LLMG_RS08935', 'LLMG_RS02815'])]}
iNF517_6
{'iNF517': [('EX_met__L_e', 'Exchange reaction'), ('Transport reaction: ILEt2r_copy2', ['LLMG_RS03390']), ('ACACT1r', ['LLMG_RS04745']), ('HB

#### P. putida

In [229]:
for i in range(len(p_putida_samples_glc)):
    print(list(p_putida_samples_glc.keys())[i])
    print(list(p_putida_samples_glc.values())[i].optimize().objective_value)

iJN1463_1
0.0
iJN1463_2
0.0
iJN1463_3
0.0
iJN1463_4
0.0
iJN1463_5
0.0
iJN1463_6
0.0
iJN1463_7
0.0
iJN1463_8
0.0
iJN1463_9
0.0
iJN1463_10
0.0


In [230]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples_glc:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples_glc[x], p_putida, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
iJN1463_2
iJN1463_3
iJN1463_4
iJN1463_5
iJN1463_6
iJN1463_7
iJN1463_8
iJN1463_9
iJN1463_10
2min 58s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [231]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples_glc[x], [p_putida], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{'iJN1463': [('IPMD', ['PP_1988']), ('3OAS181', ['PP_4175'])]}
iJN1463_2
{}
iJN1463_3
{}
iJN1463_4
{}
iJN1463_5
{}
iJN1463_6
{}
iJN1463_7
{}
iJN1463_8
{}
iJN1463_9
{}
iJN1463_10
{}
3min 6s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Test 3b: removal of 5 reactions (ribose)

In [232]:
e_coli_samples_rib = make_model_copies(e_coli_rib)
l_lactis_samples_rib = make_model_copies(l_lactis_rib)
p_putida_samples_rib = make_model_copies(p_putida_rib)

In [233]:
remove_random_reaction(e_coli_rib, e_coli_samples_rib, reactions_list=e_coli_rib_r_ids, n=5)
remove_random_reaction(l_lactis_rib, l_lactis_samples_rib, reactions_list=l_lactis_rib_r_ids, n=5)
remove_random_reaction(p_putida_rib, p_putida_samples_rib, reactions_list=p_putida_rib_r_ids, n=5)

['PSD160', 'COBALT2tex', 'MPTG', 'PNTK', 'ADSS', 'MOAT2', 'OPMEACPD', 'OPHBDC', 'DALAt2pp', 'DGK1', 'U23GAAT', 'SHCHD2', 'RPE', 'ACOAD2f', 'Zn2tex', 'DHAD1', 'VALTA', 'ECOAH2', 'GLNS', 'DHFR', 'FUM', 'COBALT2tpp', 'UAMAS', 'MG2tex', 'SO4t2pp', 'I2FE2SS2', 'ASNS2', 'TMPK', 'HSST', 'NNATr', 'PMEACPE', 'DPCOAK', 'FESR', 'GF6PTA', 'RBFSa', 'GMPS', 'LEUTAi', 'IGPS', 'MALCOAMT', 'OHPBAT', 'ICYSDS', 'ATPM', 'MNtex', 'G1SAT', 'F6PA', 'SUCCt1pp', 'CHRPL', 'ACOAD5f', 'ACLS', 'ASAD']
['TRPTRS_1', 'NAPRT', 'EX_h2s_e', 'LYSt2r', 'INSt2', 'TDPGDH_1', 'UAGCVT', 'CYTK2', 'EX_ile__L_e', 'DRPA', 'GALUi', 'RPE', 'L_LACt2r', 'GUAt2r', 'PGPP_LLA', 'DEMAT4', 'RHCYS', 'ASNTRS_1', 'ASPK', 'METt2r', 'CYTK1', 'GLUTRS_2', 'unkFol', 'EX_glu__L_e', 'ENO', 'ABUTt2r', 'EX_leu__L_e', 'ARGORNt3', 'DIACTt', 'VALTRS_1', 'GAPD', 'HOCR3', 'ALCD2x', '2MBALDt', 'aratry1', 'LEUTRS_1', 'PGGT2', 'TDMAT6', 'GRTT', 'aratyr3', 'EX_arg__L_e', 'LPGS_LLA', 'GK1', 'GLYTRS_1', 'MCMAT3', 'NH3c', 'IGPS', 'BTMAT1', 'araphe1', 'EX_gcald_e

{'iJN1463_1': <Model iJN1463 at 0x7f304677f520>,
 'iJN1463_2': <Model iJN1463 at 0x7f3056cd04f0>,
 'iJN1463_3': <Model iJN1463 at 0x7f2fcf549d90>,
 'iJN1463_4': <Model iJN1463 at 0x7f2feaa07760>,
 'iJN1463_5': <Model iJN1463 at 0x7f2fecb5a280>,
 'iJN1463_6': <Model iJN1463 at 0x7f300f760130>,
 'iJN1463_7': <Model iJN1463 at 0x7f30212913a0>,
 'iJN1463_8': <Model iJN1463 at 0x7f30209d8280>,
 'iJN1463_9': <Model iJN1463 at 0x7f30178ae190>,
 'iJN1463_10': <Model iJN1463 at 0x7f2f33f90940>}

#### E. coli

In [234]:
for i in range(len(e_coli_samples_rib)):
    print(list(e_coli_samples_rib.keys())[i])
    print(list(e_coli_samples_rib.values())[i].optimize().objective_value)

iML1515_1
0.0
iML1515_2
0.0
iML1515_3
0.0
iML1515_4
0.0
iML1515_5
0.0
iML1515_6
0.0
iML1515_7
0.0
iML1515_8
0.0
iML1515_9
0.0
iML1515_10
0.0


In [235]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples_rib[x], e_coli_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
iML1515_2
iML1515_3
iML1515_4
[<Reaction DHAD1 at 0x7f2f2f214880>, <Reaction DHFR at 0x7f2fc5c04c70>, <Reaction GLNS at 0x7f30591de910>]
iML1515_5
iML1515_6
iML1515_7
iML1515_8
iML1515_9
iML1515_10
2min 44s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [236]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples_rib[x], [e_coli_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('Transport reaction: COBALT2tex', ['b2215', 'b0929', 'b0241', 'b1377']), ('PNTK', ['b3974']), ('ADSS', ['b4177']), ('MPTG', ['b0149', 'b0089', 'b3396', 'b2519', 'b1069', 'b3208']), ('PSD160', ['b4160'])]}
iML1515_2
{}
iML1515_3
{'iML1515': [('Transport reaction: Zn2tex', ['b2215', 'b0929', 'b0241', 'b1377']), ('SHCHD2', ['b3368']), ('U23GAAT', ['b0179'])]}
iML1515_4
{'iML1515': [('DHAD1', ['b3771']), ('DHFR', ['b1606', 'b0048']), ('GLNS', ['b3870'])]}
iML1515_5
{'iML1515': [('Transport reaction: MG2tex', ['b2215', 'b0929', 'b0241', 'b1377']), ('Transport reaction: SO4t2pp', ['b2413']), ('Transport reaction: COBALT2tpp', ['b3040', 'b3816']), ('UAMAS', ['b0091'])]}
iML1515_6
{'iML1515': [('HSST', ['b4013']), ('TMPK', ['b0417']), ('NNATr', ['b0639'])]}
iML1515_7
{}
iML1515_8
{}
iML1515_9
{'iML1515': [('Transport reaction: MNtex', ['b2215', 'b0929', 'b0241', 'b1377']), ('G1SAT', ['b0154']), ('ATPM', []), ('ICYSDS', ['b2530'])]}
iML1515_10
{'iML1515': [('Transport re

#### L. lactis

In [237]:
for i in range(len(l_lactis_samples_rib)):
    print(list(l_lactis_samples_rib.keys())[i])
    print(list(l_lactis_samples_rib.values())[i].optimize().objective_value)

iNF517_1
0.0
iNF517_2
0.0
iNF517_3
None
iNF517_4
0.0
iNF517_5
0.0
iNF517_6
0.0
iNF517_7
None
iNF517_8
0.0
iNF517_9
0.0
iNF517_10
None


In [238]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples_rib[x], l_lactis_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
iNF517_2
iNF517_3
[<Reaction GALUi at 0x7f2e81ce9a90>, <Reaction L_LACt2r at 0x7f2e50637130>, <Reaction PGPP_LLA at 0x7f2e50637760>, <Reaction RPE at 0x7f2fc0e8df70>]
iNF517_4
iNF517_5
iNF517_6
[<Reaction EX_leu__L_e at 0x7f2feb7d04f0>, <Reaction VALTRS_1 at 0x7f2f2b27ad90>]
iNF517_7
[<Reaction GAPD at 0x7f2f6c568e50>, <Reaction HOCR3 at 0x7f2f6af794f0>, <Reaction ALCD2x at 0x7f2e6a2f14c0>]
iNF517_8
[<Reaction GRTT at 0x7f2e40441580>, <Reaction LEUTRS_1 at 0x7f2e40f387c0>, <Reaction PGGT2 at 0x7f2e41af3700>, <Reaction TDMAT6 at 0x7f2e40cf8b20>]
iNF517_9
iNF517_10
46.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [239]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples_rib[x], [l_lactis_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{'iNF517': [('EX_h2s_e', 'Exchange reaction'), ('Transport reaction: INSt2', []), ('Transport reaction: LYSt2r', ['LLMG_RS01980', 'LLMG_RS12445']), ('NAPRT', ['LLMG_RS07425']), ('TRPTRS_1', ['LLMG_RS00430'])]}
iNF517_2
{'iNF517': [('EX_ile__L_e', 'Exchange reaction'), ('CYTK2', ['LLMG_RS03780']), ('DRPA', ['LLMG_RS05390']), ('TDPGDH_1', ['LLMG_RS01105']), ('UAGCVT', ['LLMG_RS02630', 'LLMG_RS01685'])]}
iNF517_3
{'iNF517': [('Transport reaction: GUAt2r', ['LLMG_RS09575']), ('Transport reaction: L_LACt2r', []), ('GALUi', ['LLMG_RS05650']), ('PGPP_LLA', []), ('RPE', ['LLMG_RS04875', 'LLMG_RS11000'])]}
iNF517_4
{'iNF517': [('Transport reaction: METt2r', []), ('ASNTRS_1', ['LLMG_RS10095']), ('DEMAT4', ['LLMG_RS02820']), ('RHCYS', [])]}
iNF517_5
{}
iNF517_6
{'iNF517': [('EX_leu__L_e', 'Exchange reaction'), ('Transport reaction: DIACTt', []), ('Transport reaction: ARGORNt3', ['LLMG_RS11585', 'LLMG_RS11565']), ('Transport reaction: ABUTt2r', []), ('VALTRS_1', ['LLMG_RS12325'])]}
iNF517

#### P. putida

In [240]:
# checking functionality of the models
for i in range(len(p_putida_samples_rib)):
    print(list(p_putida_samples_rib.keys())[i])
    print(list(p_putida_samples_rib.values())[i].optimize().objective_value)

iJN1463_1
0.0
iJN1463_2
0.0
iJN1463_3
0.0
iJN1463_4
0.0
iJN1463_5
0.0
iJN1463_6
0.0
iJN1463_7
0.0
iJN1463_8
0.0
iJN1463_9
0.0
iJN1463_10
0.0


In [241]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples_rib[x], p_putida_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
iJN1463_2
iJN1463_3
iJN1463_4
iJN1463_5
iJN1463_6
iJN1463_7
iJN1463_8
iJN1463_9
iJN1463_10
3min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [242]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples_rib[x], [p_putida_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{}
iJN1463_2
{}
iJN1463_3
{}
iJN1463_4
{}
iJN1463_5
{}
iJN1463_6
{}
iJN1463_7
{}
iJN1463_8
{'iJN1463': [('Transport reaction: CU2tpp', []), ('DHORTS', ['PP_1086']), ('UAGCVT', ['PP_0964'])]}
iJN1463_9
{}
iJN1463_10
{}
3min 8s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Test 4a: removal of 10 reactions (glucose)

In [243]:
e_coli_samples_glc = make_model_copies(e_coli)
l_lactis_samples_glc = make_model_copies(l_lactis)
p_putida_samples_glc = make_model_copies(p_putida)

In [244]:
remove_random_reaction(e_coli, e_coli_samples_glc, reactions_list=e_coli_glc_r_ids, n=10)
remove_random_reaction(l_lactis, l_lactis_samples_glc, reactions_list=l_lactis_glc_r_ids, n=10)
remove_random_reaction(p_putida, p_putida_samples_glc, reactions_list=p_putida_glc_r_ids, n=10)

['OMPDC', 'MECDPDH5', 'PUNP1', 'HSST', 'NDPK8', 'UMPK', 'RPE', 'PSSA161', 'ACOAD3f', 'DAPDC', 'CTECOAI7', 'PE160abcpp', 'ANS', 'ACACT1r', 'HISTP', 'OHPBAT', '3OAR140', 'HACD6', 'GLUDy', 'EX_h_e', 'DXPRIi', 'ATPM', 'EX_pi_e', 'BIOMASS_Ec_iML1515_core_75p37M', 'SO4t2pp', 'IMPC', 'CDPMEK', 'PSSA160', 'AIRC3', 'ACACT5r', 'IPDDI', 'ACACT7r', 'EX_cl_e', 'GLYCL', 'K2L4Aabctex', 'ASPTA', 'ECOAH5', 'ASPCT', 'LPADSS', 'K2L4Aabcpp', 'PSD161', 'PRAIi', 'NH4tpp', 'ICYSDS', 'ECOAH7', 'CU2tex', 'I4FE4ST', 'PTAr', 'GHMT2r', 'PPND', 'FACOAE161', 'GLCptspp', 'EX_cobalt2_e', 'HACD4', 'ADK3', 'NDPK1', 'ADSS', 'MG2tex', 'MEOHtrpp', 'EX_mn2_e', 'EX_cu2_e', 'DHFS', 'IMPD', 'PDX5PS', 'ASNS2', 'DMATT', 'PAI2T', 'ASPK', 'ASAD', 'MOAT2', 'DGK1', 'SUCOAS', 'MALCOAMT', 'NH4tex', 'GMPS', 'AACPS7', 'APRAUR', 'NNDPR', 'NNATr', 'IPMD', 'DHAD1', 'SO4tex', 'EX_mobd_e', 'KARA1', 'SPODM', 'G5SADs', 'ACOAD4f', 'A5PISO', 'BPNT', 'E4PD', 'Kt2pp', 'I2FE2SS2', 'DHDPRy', 'G6PDH2r', 'ADNK1', 'ECOAH2', 'PRAIS', 'I2FE2ST', 'PPCDC'

{'iJN1463_1': <Model iJN1463 at 0x7f2eb7abf760>,
 'iJN1463_2': <Model iJN1463 at 0x7f2edd5147f0>,
 'iJN1463_3': <Model iJN1463 at 0x7f2f04c0e6a0>,
 'iJN1463_4': <Model iJN1463 at 0x7f2ef85fe8b0>,
 'iJN1463_5': <Model iJN1463 at 0x7f2ef85fe280>,
 'iJN1463_6': <Model iJN1463 at 0x7f2f4ca631c0>,
 'iJN1463_7': <Model iJN1463 at 0x7f2ecc466a30>,
 'iJN1463_8': <Model iJN1463 at 0x7f30131e9160>,
 'iJN1463_9': <Model iJN1463 at 0x7f300fef5220>,
 'iJN1463_10': <Model iJN1463 at 0x7f2e55142580>}

#### E. coli

In [245]:
for i in range(len(e_coli_samples_glc)):
    print(list(e_coli_samples_glc.keys())[i])
    print(list(e_coli_samples_glc.values())[i].optimize().objective_value)

iML1515_1
0.0
iML1515_2
0.0
iML1515_3
0.0
iML1515_4
0.0
iML1515_5
0.0
iML1515_6
0.0
iML1515_7
0.0
iML1515_8
0.0
iML1515_9
0.0
iML1515_10
0.0


In [246]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples_glc:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples_glc[x], e_coli, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
iML1515_2
[<Reaction ANS at 0x7f306ace8b50>, <Reaction HISTP at 0x7f2e565d3400>, <Reaction 3OAR140 at 0x7f2e431cf340>, <Reaction PE160abcpp at 0x7f2e431ba5e0>]
iML1515_3
iML1515_4
iML1515_5
iML1515_6
iML1515_7
iML1515_8
iML1515_9
iML1515_10
2min 44s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [247]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples_glc[x], [e_coli], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('OMPDC', ['b1281']), ('UMPK', ['b0171', 'b0910']), ('HSST', ['b4013']), ('DAPDC', ['b2838']), ('PSSA161', ['b2585']), ('MECDPDH5', ['b2515', 'b0684', 'b2895'])]}
iML1515_2
{'iML1515': [('EX_h_e', 'Exchange reaction'), ('ANS', ['b1264', 'b1263']), ('HISTP', ['b2022']), ('3OAR140', ['b1093']), ('PE160abcpp', ['b0914'])]}
iML1515_3
{}
iML1515_4
{'iML1515': [('EX_cl_e', 'Exchange reaction'), ('ASPTA', ['b0928', 'b4054']), ('LPADSS', ['b0182']), ('ASPCT', ['b4244', 'b4245']), ('K2L4Aabcpp', ['b0914']), ('K2L4Aabctex', ['b3201', 'b3199', 'b0054', 'b0641', 'b4261', 'b3200', 'b4262'])]}
iML1515_5
{'iML1515': [('Transport reaction: NH4tpp', ['s0001', 'b0451']), ('Transport reaction: CU2tex', ['b1377', 'b2215', 'b0241', 'b0929']), ('PPND', ['b2600']), ('PRAIi', ['b1262']), ('PSD161', ['b4160']), ('ICYSDS', ['b2530'])]}
iML1515_6
{'iML1515': [('EX_mn2_e', 'Exchange reaction'), ('EX_cobalt2_e', 'Exchange reaction'), ('Transport reaction: MG2tex', ['b1377', 'b2215', 'b0241',

#### L. lactis

In [248]:
for i in range(len(l_lactis_samples_glc)):
    print(list(l_lactis_samples_glc.keys())[i])
    print(list(l_lactis_samples_glc.values())[i].optimize().objective_value)

iNF517_1
None
iNF517_2
0.0
iNF517_3
0.0
iNF517_4
None
iNF517_5
None
iNF517_6
None
iNF517_7
0.0
iNF517_8
None
iNF517_9
0.0
iNF517_10
0.0


In [249]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples_glc:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples_glc[x], l_lactis, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
iNF517_2
iNF517_3
[<Reaction CYSt2r at 0x7f30594da7c0>, <Reaction BTMAT1 at 0x7f2febc53f40>, <Reaction EX_lac__L_e at 0x7f2f68318460>, <Reaction PROTS_LLA_v3 at 0x7f2f5e98dc70>, <Reaction RPE at 0x7f3059d35880>, <Reaction FABM2 at 0x7f3059d35a90>]
iNF517_4
iNF517_5
iNF517_6
[<Reaction ARGTRS_1 at 0x7f2eac3519a0>, <Reaction HISTRS_1 at 0x7f2ed6f3d9a0>, <Reaction PGK at 0x7f2ef8115580>, <Reaction PIabc at 0x7f2ef8115d30>, <Reaction PRAIi at 0x7f2eaea1b220>, <Reaction TKT2 at 0x7f2ea1c81940>, <Reaction TPI at 0x7f2f178e1820>, <Reaction HBUHL1 at 0x7f2efcc11d30>]
iNF517_7
iNF517_8
iNF517_9
iNF517_10
46.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [250]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples_glc[x], [l_lactis], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{}
iNF517_2
{'iNF517': [('EX_arg__L_e', 'Exchange reaction'), ('EX_for_e', 'Exchange reaction'), ('CFAS180', ['LLMG_RS10800']), ('DDMAT5', ['LLMG_RS02820']), ('DMATT', ['LLMG_RS08500']), ('DPCOAK', ['LLMG_RS03290']), ('DPMVD', ['LLMG_RS02175']), ('UAAGLS1_1', ['LLMG_RS09965', 'LLMG_RS07395'])]}
iNF517_3
{'iNF517': [('EX_lac__L_e', 'Exchange reaction'), ('EX_orn__L_e', 'Exchange reaction'), ('Transport reaction: CYSt2r', []), ('BTMAT1', ['LLMG_RS02820']), ('PROTS_LLA_v3', []), ('RPE', ['LLMG_RS04875', 'LLMG_RS11000']), ('FABM2', [])]}
iNF517_4
{}
iNF517_5
{'iNF517': [('Transport reaction: SERt2r', []), ('ACACT1r', ['LLMG_RS04745']), ('HCO3E', []), ('GLNTRS2', ['LLMG_RS00935', 'LLMG_RS00930', 'LLMG_RS00940']), ('GLUR', ['LLMG_RS06010']), ('GLYK', ['LLMG_RS05580', 'LLMG_RS05335']), ('GLYTRS_1', ['LLMG_RS07450', 'LLMG_RS07455']), ('LEUTRS_1', ['LLMG_RS08750']), ('PGSA_LLA', ['LLMG_RS11155']), ('MCMAT7', ['LLMG_RS08945'])]}
iNF517_6
{'iNF517': [('Transport reaction: H2Ot', []), ('T

#### P. putida

In [251]:
for i in range(len(p_putida_samples_glc)):
    print(list(p_putida_samples_glc.keys())[i])
    print(list(p_putida_samples_glc.values())[i].optimize().objective_value)

iJN1463_1
0.0
iJN1463_2
None
iJN1463_3
0.0
iJN1463_4
0.0
iJN1463_5
0.0
iJN1463_6
None
iJN1463_7
0.0
iJN1463_8
0.0
iJN1463_9
0.0
iJN1463_10
0.0


In [252]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples_glc:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples_glc[x], p_putida, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
iJN1463_2
iJN1463_3
iJN1463_4
iJN1463_5
iJN1463_6
iJN1463_7
iJN1463_8
iJN1463_9
iJN1463_10
3min 3s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [253]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples_glc:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples_glc[x], [p_putida], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{}
iJN1463_2
{}
iJN1463_3
{}
iJN1463_4
{}
iJN1463_5
{}
iJN1463_6
{}
iJN1463_7
{}
iJN1463_8
{}
iJN1463_9
{}
iJN1463_10
{}
3min 4s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Test 4b: removal of 10 reactions (ribose)

In [254]:
e_coli_samples_rib = make_model_copies(e_coli_rib)
l_lactis_samples_rib = make_model_copies(l_lactis_rib)
p_putida_samples_rib = make_model_copies(p_putida_rib)

In [255]:
remove_random_reaction(e_coli_rib, e_coli_samples_rib, reactions_list=e_coli_rib_r_ids, n=10)
remove_random_reaction(l_lactis_rib, l_lactis_samples_rib, reactions_list=l_lactis_rib_r_ids, n=10)
remove_random_reaction(p_putida_rib, p_putida_samples_rib, reactions_list=p_putida_rib_r_ids, n=10)

['MDH', 'ACACT2r', 'G5SD', 'EPMEACPR', 'PUNP2', 'AGPAT161', 'KDOPS', 'MPTG', 'DHFR', 'ECOAH3', 'ENO', 'MTHFC', 'FBA', 'SUCFUMtpp', 'APRAUR', 'GLUPRT', 'MOAT2', 'NDPK4', 'PANTS', 'AMAOTr', 'GLUTRS', 'GAPD', 'TPI', 'FE2tex', 'HSDy', 'LPADSS', 'ACOAD3f', 'DHORTS', 'AACPS7', 'FE2tpp', 'UDCPDP', 'PNTK', 'THDPS', 'O2tpp', 'G6PDH2r', 'PTAr', 'ACOAD6f', 'PPND', 'TKT1', 'RHCCE', 'DALAt2pp', 'I2FE2SS2', 'ILETA', 'NDPK3', 'ADNK1', 'CHRPL', 'ACOAD7f', 'ADK1', 'RBFSb', 'EX_ca2_e', 'MTHFD', 'METAT', 'PSERT', 'NH4tpp', 'PMEACPE', 'NI2tpp', 'GMPS', 'PE160abcpp', 'A5PISO', 'OMCDC', 'ACONTb', 'HCO3E', 'PRMICI', 'MG2tex', 'EX_o2_e', 'FACOAE120', 'RNTR2c2', 'GLUDy', 'P5CR', 'SPODM', 'IMPC', 'ZN2tpp', 'EX_cobalt2_e', '4HTHRK', 'AICART', 'ACOATA', 'DASYN161', 'OMPDC', 'ADCL', 'AGPAT160', 'NNDPR', 'EX_rib__D_e', 'EX_pi_e', 'ASPtpp', 'OHPBAT', 'AMPMS2', 'KDOCT2', 'HACD6', 'PERD', 'CHORM', 'ASPTA', 'OPHBDC', 'IGPS', 'HACD3', 'RNTR1c2', 'IPDDI', 'AHCYSNS', 'ASPK', 'METS', 'FUMt1pp']
['ILETRS_1', 'G3PD2', 'EX_gu

{'iJN1463_1': <Model iJN1463 at 0x7f2fb5a28c70>,
 'iJN1463_2': <Model iJN1463 at 0x7f2fb0c5ee80>,
 'iJN1463_3': <Model iJN1463 at 0x7f2f6c62b820>,
 'iJN1463_4': <Model iJN1463 at 0x7f300bd858e0>,
 'iJN1463_5': <Model iJN1463 at 0x7f2ed38ff700>,
 'iJN1463_6': <Model iJN1463 at 0x7f2e7a30d970>,
 'iJN1463_7': <Model iJN1463 at 0x7f2f02211760>,
 'iJN1463_8': <Model iJN1463 at 0x7f2f88892550>,
 'iJN1463_9': <Model iJN1463 at 0x7f300a76c3a0>,
 'iJN1463_10': <Model iJN1463 at 0x7f2fdc23ad60>}

#### E. coli

In [256]:
for i in range(len(e_coli_samples_rib)):
    print(list(e_coli_samples_rib.keys())[i])
    print(list(e_coli_samples_rib.values())[i].optimize().objective_value)

iML1515_1
0.0
iML1515_2
0.0
iML1515_3
0.0
iML1515_4
None
iML1515_5
0.0
iML1515_6
0.0
iML1515_7
0.0
iML1515_8
0.0
iML1515_9
None
iML1515_10
0.0


In [257]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples_rib[x], e_coli_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
iML1515_2
iML1515_3
iML1515_4
iML1515_5
iML1515_6
iML1515_7
iML1515_8
iML1515_9
iML1515_10
2min 42s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [258]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples_rib[x], [e_coli_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{}
iML1515_2
{}
iML1515_3
{'iML1515': [('Transport reaction: FE2tex', ['b2215', 'b0929', 'b0241', 'b1377']), ('Transport reaction: FE2tpp', ['b3040']), ('DHORTS', ['b1062']), ('LPADSS', ['b0182']), ('GLUTRS', ['b2400']), ('HSDy', ['b3940', 'b0002'])]}
iML1515_4
{'iML1515': [('Transport reaction: O2tpp', ['s0001']), ('PNTK', ['b3974']), ('RHCCE', ['b2687']), ('PPND', ['b2600']), ('UDCPDP', ['b0841', 'b3057', 'b1278']), ('THDPS', ['b0166'])]}
iML1515_5
{'iML1515': [('EX_ca2_e', 'Exchange reaction'), ('Transport reaction: DALAt2pp', ['b4208']), ('RBFSb', ['b1662']), ('ILETA', ['b3770']), ('CHRPL', ['b1096', 'b4039'])]}
iML1515_6
{}
iML1515_7
{'iML1515': [('EX_o2_e', 'Exchange reaction'), ('Transport reaction: MG2tex', ['b2215', 'b0929', 'b0241', 'b1377']), ('HCO3E', ['b0126', 'b0339']), ('P5CR', ['b0386']), ('PRMICI', ['b2024']), ('ACONTb', ['b0118', 'b1276'])]}
iML1515_8
{'iML1515': [('EX_cobalt2_e', 'Exchange reaction'), ('Transport reaction: ZN2tpp', ['b3040']), ('OMPDC', ['b

#### L. lactis

In [259]:
for i in range(len(l_lactis_samples_rib)):
    print(list(l_lactis_samples_rib.keys())[i])
    print(list(l_lactis_samples_rib.values())[i].optimize().objective_value)

iNF517_1
0.0
iNF517_2
0.0
iNF517_3
None
iNF517_4
0.0
iNF517_5
None
iNF517_6
None
iNF517_7
None
iNF517_8
None
iNF517_9
0.0
iNF517_10
0.0


In [260]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples_rib[x], l_lactis_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
iNF517_2
iNF517_3
iNF517_4
iNF517_5
[<Reaction AGAT_LLA at 0x7f2f9e846490>, <Reaction ETOHt at 0x7f2e6cebdfa0>, <Reaction DCYTD at 0x7f2ed6feda90>, <Reaction ENO at 0x7f2e905003d0>, <Reaction HODR8 at 0x7f2e88539fd0>, <Reaction FBP at 0x7f2e291d5f40>, <Reaction THRTRS_1 at 0x7f2f4fb08d30>]
iNF517_6
[<Reaction ADADir at 0x7f2e81ba9040>, <Reaction DDPA at 0x7f2e7877c790>, <Reaction GLUTRS_2 at 0x7f2eadcfd100>, <Reaction GLUTRS2 at 0x7f2eadcfd4c0>, <Reaction GLYTRS_1 at 0x7f2eadcfd0d0>, <Reaction PGI at 0x7f303cd76f40>, <Reaction PGAMT at 0x7f2ef5385a30>, <Reaction PIabc at 0x7f2ef5385100>, <Reaction TRPTRS_1 at 0x7f2e8de784f0>]
iNF517_7
iNF517_8
iNF517_9
iNF517_10
48 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [261]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples_rib[x], [l_lactis_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{}
iNF517_2
{}
iNF517_3
{'iNF517': [('EX_ura_e', 'Exchange reaction'), ('Transport reaction: L_LACt2r', []), ('DAGK_LLA', ['LLMG_RS07495']), ('NDPK4', []), ('PROTS_LLA_v3', []), ('NNATr', ['LLMG_RS01240', 'LLMG_RS07435']), ('UGLDDS1', ['LLMG_RS01860']), ('MCMAT8', ['LLMG_RS08945'])]}
iNF517_4
{}
iNF517_5
{'iNF517': [('Transport reaction: ARGt2r', []), ('Transport reaction: ETOHt', []), ('Transport reaction: SUCCt2r', []), ('AGAT_LLA', ['LLMG_RS00635']), ('DCYTD', ['LLMG_RS05395']), ('ENO', ['LLMG_RS03215']), ('HODR8', ['LLMG_RS08950']), ('FBP', ['LLMG_RS01385']), ('THRTRS_1', ['LLMG_RS10900'])]}
iNF517_6
{'iNF517': [('ADADir', ['LLMG_RS03940']), ('DDPA', ['LLMG_RS00665', 'LLMG_RS06200']), ('GLUTRS_2', ['LLMG_RS11690']), ('GLUTRS2', ['LLMG_RS11690']), ('GLYTRS_1', ['LLMG_RS07450', 'LLMG_RS07455']), ('PGI', ['LLMG_RS12290']), ('PGAMT', ['LLMG_RS02290']), ('PIabc', ['LLMG_RS09515', 'LLMG_RS09500', 'LLMG_RS09495', 'LLMG_RS09505', 'LLMG_RS09510', 'LLMG_RS09490']), ('TRPTRS_1', ['LL

#### P. putida

In [262]:
# checking functionality of the models
for i in range(len(p_putida_samples_rib)):
    print(list(p_putida_samples_rib.keys())[i])
    print(list(p_putida_samples_rib.values())[i].optimize().objective_value)

iJN1463_1
0.0
iJN1463_2
0.0
iJN1463_3
0.0
iJN1463_4
0.0
iJN1463_5
0.0
iJN1463_6
None
iJN1463_7
0.0
iJN1463_8
0.0
iJN1463_9
0.0
iJN1463_10
0.0


In [263]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples_rib:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples_rib[x], p_putida_rib, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
iJN1463_2
iJN1463_3
iJN1463_4
iJN1463_5
iJN1463_6
iJN1463_7
iJN1463_8
iJN1463_9
iJN1463_10
2min 58s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [265]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples_rib:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples_rib[x], [p_putida_rib], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{}
iJN1463_2
{}
iJN1463_3
{}
iJN1463_4
{}
iJN1463_5
{}
iJN1463_6
{}
iJN1463_7
{}
iJN1463_8
{}
iJN1463_9
{}
iJN1463_10
{}
3min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
